In [74]:
import cv2
import os
from custom_dset_new import train_val_test_split
import numpy as np
from tqdm import tqdm

In [9]:
data_dir = '/Users/ruoyangzhang/Documents/PythonWorkingDirectory/Assignment_data/images'

In [10]:
train_data, val_data, test_data = train_val_test_split(data_dir, train_split=0.8, val_split=0.2, test_split = 0)

In [51]:
img = cv2.imread(list(train_data.keys())[0])
b, g, r = cv2.split(img)
rgb_img = cv2.merge([r, g, b])

In [52]:
rgb_img.shape

(28, 28, 3)

In [22]:
rgb_img[0]

array([[ 37,  47,  94],
       [ 50,  78, 106],
       [ 56,  78, 109],
       [ 40,  43,  98],
       [ 60,  83,  90],
       [ 58,  59,  91],
       [ 48,  59,  94],
       [ 81,  85,  97],
       [104,  98,  98],
       [ 95,  94,  94],
       [ 61,  76,  96],
       [ 65,  95,  87],
       [ 63,  84,  97],
       [ 35,  51,  93],
       [ 25,  36,  86],
       [ 29,  35,  91],
       [ 75,  82, 102],
       [107, 127, 113],
       [ 85, 101, 112],
       [ 54,  57,  99],
       [ 55,  70,  97],
       [ 65,  88,  97],
       [ 61,  80,  97],
       [ 52,  60,  87],
       [ 50,  60,  85],
       [ 37,  56,  84],
       [ 25,  47,  85],
       [ 22,  34,  82]], dtype=uint8)

In [35]:
rgb_img.shape = (1,28*28*3)

In [36]:
rgb_img

array([[ 37,  47,  94, ...,  68,  84, 101]], dtype=uint8)

In [40]:
np.concatenate((rgb_img, rgb_img, rgb_img))

array([[ 37,  47,  94, ...,  68,  84, 101],
       [ 37,  47,  94, ...,  68,  84, 101],
       [ 37,  47,  94, ...,  68,  84, 101]], dtype=uint8)

In [66]:
ordered_train_dirs = [dir for dir in sorted(list(train_data.keys())) if os.path.split(dir)[-1] != '.DS_Store']

In [68]:
ordered_train_dirs[0]

'/Users/ruoyangzhang/Documents/PythonWorkingDirectory/Assignment_data/images/barren_land/10.png'

In [71]:
def convert_to_vector(img_dir):
    img = cv2.imread(img_dir)
    b, g, r = cv2.split(img)
    rgb_img = cv2.merge([r, g, b])
    rgb_img.shape = (1, 28*28*3)
    return(rgb_img)

In [78]:
input_images = np.array([convert_to_vector(dir) for dir in tqdm(ordered_train_dirs)])


 10%|█         | 26059/258588 [00:14<02:07, 1828.52it/s]


 22%|██▏       | 55621/258588 [00:28<01:42, 1971.24it/s]


 32%|███▏      | 82476/258588 [00:42<01:30, 1952.92it/s]


 43%|████▎     | 111488/258588 [00:56<01:14, 1982.73it/s]


 53%|█████▎    | 138101/258588 [01:10<01:01, 1971.21it/s]


 65%|██████▍   | 166842/258588 [01:23<00:46, 1989.72it/s]


 75%|███████▍  | 192877/258588 [01:37<00:33, 1975.37it/s]


 86%|████████▌ | 221931/258588 [01:51<00:18, 1992.42it/s]


 96%|█████████▌| 248430/258588 [02:05<00:05, 1984.05it/s]


100%|██████████| 258588/258588 [02:10<00:00, 1979.99it/s]

In [82]:
input_images.shape = (258588, 2352)

In [83]:
input_images.shape

(258588, 2352)

In [84]:
input_images

array([[158, 152, 125, ..., 123, 142, 111],
       [202, 187, 160, ..., 208, 194, 162],
       [151, 132, 107, ..., 157, 134, 110],
       ...,
       [ 49,  72,  93, ..., 100, 128, 117],
       [ 39,  55,  82, ...,  33,  58,  77],
       [104, 142, 124, ...,  65, 103, 106]], dtype=uint8)

In [87]:
labels = np.array([train_data[dir] for dir in ordered_train_dirs])

In [90]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [96]:
clf=RandomForestClassifier(n_estimators=100, n_jobs = 5)

In [97]:
clf.fit(input_images,labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=5,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)